# UofT FASE ML Bootcamp
#### Tuesday August 20, 2019
#### Decision Trees & Random Forests - Lab 1, Day 2 
#### Teaching team: Elias Khalil, Kyle E. C. Booth, and Alex Olson 
##### Lab author: Kyle E. C. Booth, kbooth@mie.utoronto.ca

So far, we've learned about nearest neighbors, support vector machines (SVM) and regression techniques. In this lab, we will be introducing *decision tree and forests*. We will introduce the notion of a decision tree, extend this to random forests, and then investigate some state-of-the-art tree-based methods for machine learning. Let's get started!

In [1]:
import numpy as np
import sklearn

### Decision Trees

Decision trees are popular supervised learning methods used for classification and regression. The tree represents a series of simple decision rules that predict the target when the feature vector is passed through them. Decision trees are easy to understand, can be visualized nicely, require very little data preparation (e.g., we don't need to scale features), and the trained model can be explained easily to others post priori (as opposed to other *black box* methods that are difficult to communicate).  

###### Example
Suppose you wanted to wanted to design a simple decision tree for whether (or not) you buy a used car. You might develop something like the following:

<img src="img/decision-tree.gif" width="500"/>

**YOUR TURN:** Let's say you're browsing Kijiji and come across a used car that: has been road tested, has high mileage, and is a recent year/model.
* According to your decision tree model, should you buy this car or not? ____________________________
* Will you buy any cars that haven't been road tested (if you follow your model)? ___________________________________

Obviously this tree may not be ideal, depending on the situation. For example, you could have a road tested car of a recent year with 2,000,000 km's on it and the model is telling you to buy! (But, you probably shouldn't)

##### Titanic Survivor Dataset

In this lab, we will be exploring the use of decision trees in the context of Kaggle's famous **Titanic dataset**. Each row in the data represents a passenger, detailing various characteristics about them (i.e., the features), and also details whether or not the passenger survived the disaster.

Let's load the data and take a look at it.

To get the data into a manageable format, we're going to use the [Pandas](https://pandas.pydata.org/) library, a popular library for data manipulation and analysis. While we won't be providing a full Pandas tutorial, we will provide some insight into key functions.  

In [3]:
import pandas as pd
data = pd.read_csv('data/titanic.csv', index_col=0)
data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


The above cell used the `.read_csv()` function in Pandas to pull in the comma-separated Titanic survivor data (this is a very powerful/popular function!). The `.head()` allows us to conveniently take a glance at the first 5 rows (along with the header). 

We can see that, along with the target 'Survived', we have a number of features including the passenger name, sex, age, fare, cabin, etc. We can do a bit of simple *exploratory data analysis* (EDA) to get a better feel for the dataset.

In [4]:
print ("Passengers, features: ", data.shape)
print ("Survived: ", data[data["Survived"]==1].shape[0], ", Didn't: ", data[data["Survived"]==0].shape[0])
print ("Female: ", data[data["Sex"]=="male"].shape[0], ", Male: ", data[data["Sex"]=="female"].shape[0])
print ("\n Missing values by feature: \n", data.isna().sum())

Passengers, features:  (891, 11)
Survived:  342 , Didn't:  549
Female:  577 , Male:  314

 Missing values by feature: 
 Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64


As you can see, we can use Pandas to conveniently summarize key aspects of the dataset such as the number of passengers, features, survived/didn't, and their gender. We are also able to identify the number of missing values per feature in the dataset. 

To accomplish this, we used Pandas flexible indexing capability. The syntax `data[data[col]==val]` allows us to return the subset of rows in `data` where column `col` takes on value `val`. Very powerful!

As you may have suspected, the dataset we're using is actually a subset of the total Titanic data. In reality, there were actually 3,547 passengers while the data we're working with only concerns 891 of them.

**YOUR TURN:**
Using similar syntax, answer the following questions about the data:
* In the dataset, what is the passenger survival rate? ____________________________ 
* How many passengers paid more than $10 for fare? ____________________________
* How many passengers had a passenger class (Pclass) of 3? ________________________
* With some discussion/exploration and try to determine what features might be the most relevant to passenger survival.

In [114]:
## Your code here
## 
## 

##### Data Preparation: Categorical -> Numerical Mapping

Before we can fit sklearn decision trees to our data, we first need to convert all of the categorical variables (e.g., gender) numerical values. Categoricals with unique values (like name and ticket #) can be removed from the dataset entirely as don't suspect they will contribute to the model.

We can do the required preparation as follows:

In [115]:
from sklearn import preprocessing
data = data.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

le = preprocessing.LabelEncoder()
le.fit(data['Sex'])
data['Sex'] = le.transform(data['Sex'])

data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
PassengerId,,,,,,,
1,0,3,1,22.0,1,0,7.2500
2,1,1,0,38.0,1,0,71.2833
3,1,3,0,26.0,0,0,7.9250
4,1,1,0,35.0,1,0,53.1000
5,0,3,1,35.0,0,0,8.0500


In the above cell, we dropped a number of columns we don't suspect will be correlated with the target (*Note: we probably should have been a bit more rigorous about this!*). Then we used the `LabelEncoder()` within sklearn that can fit a numbering scheme to a categorical feature (i.e., sex). We can see in the new dataset, sex takes on a value of 0 (female) or 1 (male).

##### Model Development

OK! Let's get to developing some tree-based models to predict passenger survival. We will start with simple decision trees and develop more complex models from there. Our first step, as in previous labs, is to split our data into a training set and a test set (unseen data). We will then use k-folds cross validation on the training set to try and get the best performing model before finally applying it to the test data.

Let's import sklearn's decision tree classifer and split the data. We also need 

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree

target_data = data["Survived"]
feature_data = data.iloc[:, data.columns != "Survived"]

X_train, X_test, y_train, y_test = train_test_split(feature_data, target_data, test_size=0.3, random_state=0)

**YOUR TURN:**
* How many samples are in the training set? _______________________
* How many samples are in the test set? _______________________
* What are the survival rates in each of the datasets? ______________________

In [117]:
## Your code here
##
##

##### Dealing with Missing Data: Imputation

Before we can fit our decision tree to our training data, we can conduct *imputation* to replace missing values with the mean/median/mode value in the column. For this exercise we will conduct mode imputation (i.e., the most common value in the column). 

It's important that you don't impute your data using some of the test data! This is an example of *information leak* where your test data is leaking into your training data. As such, we will fit our missing data imputer to our training data only.

In [118]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values=np.nan, strategy='most_frequent')

imp.fit(X_train)
X_train = imp.transform(X_train)

So we've got our data prepared, let's fit a decision tree to our training data. Remember, the pipeline for model development in sklearn is **initialize->fit->predict**.

In [119]:
clf = tree.DecisionTreeClassifier()
scores = cross_val_score(clf, X_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.76 (+/- 0.05)


In the above cell, we defined a Decision Tree classifier and used 5-folds cross validation to evaluate it. The resulting accuracy was ~75%.

**YOUR TURN:**
* If you designed a naive classifier that simply guessed 'did not survive' on the training set, how would it perform? ________________________
* Is this better or worse than our decision tree? ____________________________

In [110]:
## Your code here
##
##

##### Feature Importances

One thing we can do is take a look at the relative feature importances of the trained decision tree classifier. This will give us an idea of what the model thinks is more/less important for properly predicting the target. We can also visualize the actual tree iteself using [graphViz](https://www.graphviz.org/), but we won't be covering that today.

Let's look at the feature importances for a model on the training data:

In [126]:
clf.fit(X_train, y_train)
print ("Features: ", feature_data.columns)
print ("Relative feature importances: ", clf.feature_importances_)

Features:  Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')
Relative feature importances:  [0.10112378 0.31202983 0.23662049 0.07834671 0.03674885 0.23513035]


As we can see, the tree is placing a higher importance on Age, # of Siblings/Spouses Aboard (SibSp), and Fare paid. These are interesting observations and may indicate that children/families had a higher rate of survival; we would have to conduct a more thorough EDA to be certain. 

### Random Forests, Ada Boost, and Gradient Boosting

Evidently our cross-validated decision tree accuracy wasn't that great at 75%. Let's investigate some more advanced tree models that you have learned about and see if we can improve our performance. We will be using the following models in addition to our decision tree classifier:

* Scikitlearn Random Forest classifier
* Scikitlearn Adaboost classifier
* Scikitlearn Gradient boosting classifier

In [129]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

Now, let's see how they perform!

In [131]:
clf_random = RandomForestClassifier()
clf_ada = AdaBoostClassifier()
clf_gradient = GradientBoostingClassifier()

scores = cross_val_score(clf, X_train, y_train, cv=5)
scores_random = cross_val_score(clf_random, X_train, y_train, cv=5)
scores_ada = cross_val_score(clf_ada, X_train, y_train, cv=5)
scores_gradient = cross_val_score(clf_gradient, X_train, y_train, cv=5)

print("Decision tree accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("Random forest accuracy: %0.2f (+/- %0.2f)" % (scores_random.mean(), scores_random.std() * 2))
print("Adaboost accuracy: %0.2f (+/- %0.2f)" % (scores_ada.mean(), scores_ada.std() * 2))
print("Gradient boosting accuracy: %0.2f (+/- %0.2f)" % (scores_gradient.mean(), scores_gradient.std() * 2))

Decision tree accuracy: 0.76 (+/- 0.06)
Random forest accuracy: 0.79 (+/- 0.08)
Adaboost accuracy: 0.80 (+/- 0.04)
Gradient boosting accuracy: 0.83 (+/- 0.02)


We can see that each of the more sophisticated tree/forest methods improves upon the initial decision tree accuracy in terms of cross-validated accuracy, with gradient boosting providing the best result.

Let's see how the gradient boosted method performs on the hold-out test set.

In [134]:
from sklearn.metrics import accuracy_score

imp = Imputer(missing_values=np.nan, strategy='most_frequent')

imp.fit(X_test)
X_test = imp.transform(X_test)

clf_gradient.fit(X_train, y_train)
accuracy = accuracy_score(y_test, clf_gradient.predict(X_test))

print ("Test set accuracy: ", accuracy * 100, "%")

Test set accuracy:  82.08955223880598 %


First, we imputed the missing values in the test set (as we had done for the training set) and then we applied our gradient boosting-based classifier (as trained on the training data). We yielded an 82% accuracy; not bad!

**YOUR TURN:**
* What features did the gradient boosting algorithm find the most important? __________________
* What is the test set accuracy if, instead, you used the Adaboost algorithm? __________________

##### Congratulatons! 

You're finished this lab. On to the next!